In [1]:
import pandas as pd
from openai import OpenAI

import os
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

df = pd.read_json('../data/processed_data.jsonl', lines=True)

In [2]:
df.head()

,title,average_rating,rating_number,features,description,price,store,thumbnail,product_spec
0,Mento Streamtail,2.0,1,"[Thermoplastic Rubber sole, High Density Premi...",[Slip on the Women's Mento and you're ready to...,29.81,Guy Harvey,https://m.media-amazon.com/images/I/31P-uHUUIX...,Mento Streamtail Thermoplastic Rubber sole Hig...
1,RONNOX Women's 3-Pairs Bright Colored Calf Com...,4.3,3032,"[Pull On closure, Size Guide: ""S"" fits calf 10...",[Ronnox Calf Sleeves - Allowing Your Body to P...,17.99,RONNOX,https://m.media-amazon.com/images/I/51CqMDJOOD...,RONNOX Women's 3-Pairs Bright Colored Calf Com...
2,LYCKYY Women's Tie Dye Sweatshirt Crewneck Lon...,3.7,52,[Pull On closure],[Tie dye shirts for Women long sleeve crewneck...,9.99,LYCKYY,https://m.media-amazon.com/images/I/41DW4uwWLQ...,LYCKYY Women's Tie Dye Sweatshirt Crewneck Lon...
3,Sexyshine Women's Casual Fall Knit Long Sleeve...,3.6,7,"[Cotton Blend, Asian Size,Smaller than US Size...",[Sexyshine Women's Casual Fall Knit Long Sleev...,26.99,Sexyshine,https://m.media-amazon.com/images/I/51Cijpzflv...,Sexyshine Women's Casual Fall Knit Long Sleeve...
4,Result Core Men's Soft Shell II Gilet Red 3XL,5.0,1,"[Polyester,Microfleece,Elastane]",[],53.95,Result Core,https://m.media-amazon.com/images/I/41vLlRDu6V...,Result Core Men's Soft Shell II Gilet Red 3XL ...


In [3]:
import tiktoken
import concurrent.futures
from tqdm import tqdm

from tenacity import retry, wait_random_exponential, stop_after_attempt

EMBEDDING_MODEL = "text-embedding-3-small"

# Simple function to take in a list of text objects and return them as a list of embeddings
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(30))
def get_embeddings(input):
    response = client.embeddings.create(
        input=input,
        model=EMBEDDING_MODEL
    ).data
    return [data.embedding for data in response]


# Splits an iterable into batches of size n.
def batchify(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx : min(ndx + n, l)]
     

# Function for batching and parallel processing the embeddings
def embed_text(
    text,
    batch_size=64,
    num_workers=12,
    max_context_len=8191,
):
    # Encode the text, truncating to max_context_len
    encoding = tiktoken.get_encoding("cl100k_base")
    encoded_text = [
        encoded_article[:max_context_len] for encoded_article in encoding.encode_batch(text)
    ]
    # Embed the text
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        
        futures = [
            executor.submit(get_embeddings, text_batch)
            for text_batch in batchify(encoded_text, batch_size)
        ]

        with tqdm(total=len(encoded_text)) as pbar:
            for _ in concurrent.futures.as_completed(futures):
                pbar.update(batch_size)

        embeddings = []
        for future in futures:
            data = future.result()
            embeddings.extend(data)

        return embeddings
    

In [4]:
product_names = df['title'].astype(str).tolist()

embeddings = embed_text(product_names)

df['embedding'] = embeddings


50304it [09:42, 86.42it/s]                             


RetryError: RetryError[<Future at 0x177ed71a0 state=finished raised BadRequestError>]

In [10]:
df.to_json('../data/processed_data_with_title_embeddings.jsonl', orient='records', lines=True)